# Baseball Gestión de Multicolinealidad para Modelos de Regresión usando PCA

## Objetivos de Aprendizaje

Utilizar PCA (Principal Components-Decomposition Analysis) para eliminar multi-colinealidad de modelos
    * identificación de multicolinealidad en las variables del modelo
    * entrenar e interpretar PCA
    * ejecutar regresión en componentes principales

## Imports

In [ ]:
import pandas as pd
import statsmodels.api as sms
import sklearn.decomposition as dcmp

In [ ]:
%matplotlib inline

## Obtención de Data y Subset de Data

In [ ]:
# lectura de archivo csv y generación del dataframe
df = pd.read_csv('../data/baseball_data.csv')

# generación de subset dataframe eliminando filas con valores NULLs
bix = df.notnull().all(axis=1)
df = df[bix]

## Primer Modelo de Regresión utilizando Todas las Variables

El objetivo de esta primera etapa es poder analizar como se relacionan todas las variables con el target (variable/atributo objetivo). Desafortunadamente, esto falla debido a la alta co-relación entre algunas variables.

In [ ]:
model = sms.OLS(df.salary_in_thousands_of_dollars, sms.add_constant(df.iloc[:, 1:]))

In [ ]:
result = model.fit()

In [ ]:
print(result.summary())

#### Notas:
    * Con este enfoque, logramos obtener un R^2 bastante alto desde el inicio 0.71, esto no siempre es bueno.
    * Mediante este resumen estadístico, se puede comenzar a observar cuales variables son 
    potencialmente significativas
        - on_base_percentage
        - number_of_runs
        - number_of_runs_batted_in
        - number_of_strike_outs
        - number_of_stolen_bases
        - indicator_of_free_agency_eligibility
        - indicator_of_free_agent_in_1991_1992
        - indicator_of_arbitration_eligibility
        - indicator_of_arbitration_in_1991_1992
    * Aun hay bastantes variables, como podemos discernir cuales usar y no usar en el modelo?
    * Warning [2] indica que probablemente existe una alta colinelidad entre algunas variables.

## Analizar Correlaciones entre las Variables

Dentro de los primeros analisis que debiesemos hacer, es el de analizar la correlación entre variables.

In [ ]:
df.corr().style.background_gradient(cmap='Blues')

Al usar .style.background_gradient(cmap='viridis') al final del método corr(), es posible añadir un estilo a la matriz de correlación, lo cual ayuda a la visualización de variables con alta correlación.

Notas:
    * existe alta correlación entre sueldo y número de variables que no son significativas en el primer modelo.
    * hay una baja correlación entre variables que en el primer modelo parecen significativas y sueldo.
    * hay una alta correlación entre sueldo y algunas variables que en el primer modelo no son consideradas como significativas.
    * Algunas correlaciones entre variables pueden estar probocando imprecisión en el primer modelo.

## Gestión para la Correlación de Variables

###### Crear y ajustar Objeto PCA

In [ ]:
pca = dcmp.PCA()
pca.fit(df.iloc[:, 1:])

###### Análisis de Componentes

Si el modelo busca ser interpretado y no solo generar predicciones, necesitamos entregar interpretación de variables.

En los comandos de abajo, revisamos los PCs y observamos:
    - las variables con nombre "number_of_xxxx" donde xxxx es una actividad relacionada con el bateo, están altamente cargadas en PC1.
    - number_of_hits y number_of_runs están cargadas altamente en positivo, mientras los home_runs, rbis, walks, y strike_outs están cargadas altamente hacia valores negativos para PC2.
    
Nuestra interpretación para los primeros dos PCs (PC1 y PC2), debiese ser:
    - PC1 describe mayoritariamente el comportamiento de las variables "number_of_xxxx," siendo xxxx actividades durante el bateo.
    - PC2 describe variables relacionadas con la eficiencia del bateo.
        * aquí la mayor carga positiva se encuentra en number_of_hits y la mayor carga negativa en number_of_strike_outs (the loadings are "competing")

In [ ]:
pd.DataFrame(pca.components_.T, 
                columns=['PC' + str(i+1) for i in range(pca.components_.shape[1])],
                index=df.iloc[:, 1:].columns)

###### Analisis de Scree Plot

In [ ]:
pd.Series(pca.explained_variance_ratio_).plot(title='Scree Plot');

###### Obtener Transformación de Data

In [ ]:
transformed = pca.fit_transform(df.iloc[:, 1:])
reduced = transformed[:, :2]
data = pd.DataFrame(reduced, columns=['p1', 'p2'])

###### Ajustes de Regresión sobre la data transformada

In [ ]:
model = sms.OLS(df.salary_in_thousands_of_dollars.reset_index().drop('index', axis=1), sms.add_constant(data))

In [ ]:
results = model.fit()

In [ ]:
print(results.summary())